# S04 T01: Transformació Registre Log amb Regular expressions
## Luis Pardina - Data Science - 21 abril 2022

***Exercici 1***: Estandaritza, identifica i enumera cada un dels atributs / variables de l'estructura de l'arxiu "Web_access_log-akumenius.com" que trobaràs al repositori de GitHub "Data-sources".#

In [35]:
import numpy as np
import pandas as pd
import re

He trobat a internet una explicació del format característic dels *web access logs*: (www.sumologic.com/blog/apache-access-log/).

Seguint aquesta guia, observo i identifico el contingut de cada registre d'aquest *web access log*. Veig que hi han fins a 11 camps, separats per espais en blanc (host, ip_client, user, user_id, date_and_time, request_type_and_resource, HTTP_status_code, size, HTTP_referer, User_Agent, ?).

Ara bé, alguns d'aquests camps contenen dintre seu espais en blanc. En aquests casos o bé el camp està delimitat per square brackets [ ] (és el cas de date_and_time) o bé per cometes " " (és el cas de request_type_and_resource, HTTP_referer, i User_Agent).

Llavors l'optim seria utilitzar per a generar el dataframe un delimitador de les columnes basat en una regex que segueixi aquesta logica. Buscant per internet a (https://linuxtut.com/en/79cd4fad1d7ea1fed12b/) he trobat aquesta regex \s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\])

La regex es prou complexa però bàsicament busca els espais en blanc \s tot fent un *lookahead* mirant el que segueix a continuació per a no comptar com a delimitador quan està dins de square_brackets o cometes.

In [36]:
#creo una llista amb els noms de les columnes
noms = ['host','ip_client','user','user_id','date_time', 'request_resource', 'status', 'size','referer' ,'user_agent', '??']

#regex que ens permet delimitar camps de cada registre de l'arxiu
delim = r'\s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\])' 

def trec_ext(x):                                         #funció per a treure cometes i square brackets
    return x[1:-1]            
neteja = {'date_time':trec_ext, 'request_resource': trec_ext, 'referer':trec_ext, 'user_agent':trec_ext}

bitacora = pd.read_csv('Web_access_log-akumenius.com.txt', sep=delim, names=noms, converters=neteja, na_values='-', engine='python')

In [37]:
bitacora.shape

(261873, 11)

In [38]:
bitacora.head(5)

,host,ip_client,user,user_id,date_time,request_resource,status,size,referer,user_agent,??
0,localhost,127.0.0.1,NaN,NaN,23/Feb/2014:03:10:31 +0100,OPTIONS * HTTP/1.0,200,NaN,NaN,Apache (internal dummy connection),VLOG=-
1,localhost,127.0.0.1,NaN,NaN,23/Feb/2014:03:10:31 +0100,OPTIONS * HTTP/1.0,200,NaN,NaN,Apache (internal dummy connection),VLOG=-
2,localhost,127.0.0.1,NaN,NaN,23/Feb/2014:03:10:31 +0100,OPTIONS * HTTP/1.0,200,NaN,NaN,Apache (internal dummy connection),VLOG=-
3,localhost,127.0.0.1,NaN,NaN,23/Feb/2014:03:10:31 +0100,OPTIONS * HTTP/1.0,200,NaN,NaN,Apache (internal dummy connection),VLOG=-
4,localhost,127.0.0.1,NaN,NaN,23/Feb/2014:03:10:31 +0100,OPTIONS * HTTP/1.0,200,NaN,NaN,Apache (internal dummy connection),VLOG=-


In [39]:
bitacora.tail(5)

,host,ip_client,user,user_id,date_time,request_resource,status,size,referer,user_agent,??
261868,www.akumenius.com,5.255.253.53,NaN,NaN,02/Mar/2014:03:05:39 +0100,GET / HTTP/1.1,200,7528.0,NaN,Mozilla/5.0 (compatible; YandexBot/3.0; +http:...,VLOG=-
261869,www.akumenius.com,74.86.158.107,NaN,NaN,02/Mar/2014:03:09:52 +0100,HEAD / HTTP/1.1,200,NaN,NaN,Mozilla/5.0+(compatible; UptimeRobot/2.0; http...,VLOG=-
261870,localhost,127.0.0.1,NaN,NaN,02/Mar/2014:03:10:18 +0100,OPTIONS * HTTP/1.0,200,NaN,NaN,Apache (internal dummy connection),VLOG=-
261871,localhost,127.0.0.1,NaN,NaN,02/Mar/2014:03:10:18 +0100,OPTIONS * HTTP/1.0,200,NaN,NaN,Apache (internal dummy connection),VLOG=-
261872,localhost,127.0.0.1,NaN,NaN,02/Mar/2014:03:10:18 +0100,OPTIONS * HTTP/1.0,200,NaN,NaN,Apache (internal dummy connection),VLOG=-


***Exercici 2***: Neteja, preprocesa, estructura i transforma (dataframe) les dades del registre d'Accés a la web. 

1. Inspecciono si hi ha columnes completamente buides. 
2. Veig que la columna que he anomenat 'user' no conté res (té tants camps buits com registres te el dataframe 261973), i l'elimino. 
3. Examino qué conté la columna que he anomenat '??', veig que només conté la string *VLOG=-*, i l'elimino.

In [40]:
bitacora.isnull().sum()

host                     0
ip_client                0
user                261873
user_id             261846
date_time                0
request_resource        37
status                   0
size                 42335
referer              99547
user_agent             219
??                       0
dtype: int64

In [41]:
bitacora['??'].unique()

array(['VLOG=-'], dtype=object)

In [42]:
bitacora.drop(['user', '??'], axis=1, inplace=True)      #elimino totes dues columnes sense informació rellevant

4. Creo tres noves columnes que enmagatzemaran la data, hora i fus horari de visita, informació que està dins la columna 'date_time'. Utilitzo el mètode .insert() per a insertar les noves columnes. 
5. Esborro la columna 'date_time'.

In [43]:
bitacora.insert(3, "date", bitacora.date_time.str.slice(0,11))                      #la data són sempre aquests 11 caràcters
bitacora.insert(4, "time", bitacora.date_time.str.slice(12,20))                     #l'hora són sempre aquests 8 caràcters
bitacora.insert(5, "standardtime", bitacora["date_time"].str.split(" ",1).str[1])   #el fus està després del primer blanc

bitacora.drop("date_time",axis=1, inplace=True)


6. Faig un split de la columna 'request_resource' en les tres informacions que conté: el tipus de requeriment, el recurs requerit i el protocol. Aquestes tres informacions estan separades per espais en blanc
7. Esborro la columna original 'request_resource' per a eliminar la informació duplicada    

In [44]:
bitacora.insert(6, "request_type", bitacora["request_resource"].str.split(" ").str[0])
bitacora.insert(7, "resource", bitacora["request_resource"].str.split(" ").str[1])
bitacora.insert(8, "protocol", bitacora["request_resource"].str.split(" ").str[2])

bitacora.drop("request_resource", axis=1, inplace=True)


In [45]:
bitacora.head(5)

,host,ip_client,user_id,date,time,standardtime,request_type,resource,protocol,status,size,referer,user_agent
0,localhost,127.0.0.1,NaN,23/Feb/2014,03:10:31,+0100,OPTIONS,*,HTTP/1.0,200,NaN,NaN,Apache (internal dummy connection)
1,localhost,127.0.0.1,NaN,23/Feb/2014,03:10:31,+0100,OPTIONS,*,HTTP/1.0,200,NaN,NaN,Apache (internal dummy connection)
2,localhost,127.0.0.1,NaN,23/Feb/2014,03:10:31,+0100,OPTIONS,*,HTTP/1.0,200,NaN,NaN,Apache (internal dummy connection)
3,localhost,127.0.0.1,NaN,23/Feb/2014,03:10:31,+0100,OPTIONS,*,HTTP/1.0,200,NaN,NaN,Apache (internal dummy connection)
4,localhost,127.0.0.1,NaN,23/Feb/2014,03:10:31,+0100,OPTIONS,*,HTTP/1.0,200,NaN,NaN,Apache (internal dummy connection)


In [46]:
bitacora.tail(5)

,host,ip_client,user_id,date,time,standardtime,request_type,resource,protocol,status,size,referer,user_agent
261868,www.akumenius.com,5.255.253.53,NaN,02/Mar/2014,03:05:39,+0100,GET,/,HTTP/1.1,200,7528.0,NaN,Mozilla/5.0 (compatible; YandexBot/3.0; +http:...
261869,www.akumenius.com,74.86.158.107,NaN,02/Mar/2014,03:09:52,+0100,HEAD,/,HTTP/1.1,200,NaN,NaN,Mozilla/5.0+(compatible; UptimeRobot/2.0; http...
261870,localhost,127.0.0.1,NaN,02/Mar/2014,03:10:18,+0100,OPTIONS,*,HTTP/1.0,200,NaN,NaN,Apache (internal dummy connection)
261871,localhost,127.0.0.1,NaN,02/Mar/2014,03:10:18,+0100,OPTIONS,*,HTTP/1.0,200,NaN,NaN,Apache (internal dummy connection)
261872,localhost,127.0.0.1,NaN,02/Mar/2014,03:10:18,+0100,OPTIONS,*,HTTP/1.0,200,NaN,NaN,Apache (internal dummy connection)


8. Ara amb el mètode .value_counts() vaig a presentar el recompte de valors únics a les principals columnes. Es pot veure que hi ha hagut visites des de 2921 ip's diferents, entre el 25 de febrer i el 2 de març del 2014.

In [47]:
bitacora['host'].value_counts()

www.akumenius.com     232300
test.akumenius.com     14610
localhost              14127
akumenius.com            742
akumenius.es              94
Name: host, dtype: int64

In [48]:
bitacora['ip_client'].value_counts()

66.249.76.216      46382
80.28.221.123      14725
127.0.0.1          13892
217.125.71.222      5201
66.249.75.148       3558
                   ...  
84.123.150.27          1
217.130.150.116        1
202.46.52.23           1
216.151.130.170        1
206.198.5.33           1
Name: ip_client, Length: 2921, dtype: int64

In [49]:
bitacora['user_id'].value_counts()

clarcat    27
Name: user_id, dtype: int64

In [50]:
bitacora['date'].value_counts()

25/Feb/2014    55798
24/Feb/2014    48850
27/Feb/2014    41513
23/Feb/2014    40536
26/Feb/2014    36473
28/Feb/2014    19570
01/Mar/2014    17749
02/Mar/2014     1384
Name: date, dtype: int64

In [51]:
bitacora['request_type'].value_counts()

GET                  229482
POST                  16669
OPTIONS               13892
HEAD                   1788
\x80w\x01\x03\x01         5
Name: request_type, dtype: int64

In [52]:
bitacora['resource'].value_counts().head(25)

*                                                                       13892
/destinos-get                                                            8115
/                                                                        4058
/hotel-list-data/                                                        2342
/modules/raton/views/themes/bcoos/images/boto_home_reserva.png           1754
/includes/images/uploaded/logo.png                                       1740
/modules/raton/views/themes/bcoos/images/boto_home_planea.png            1738
/modules/raton/views/themes/bcoos/images/boto_home_elige.png             1731
/raton-search                                                            1694
/hotel-list                                                              1666
/libraries/jquery/jquery-1.4.2.min.js                                    1646
/modules/raton/views/themes/bcoos/images/buscadores/calendaricon.png     1645
/includes/css/style.css                                         

In [53]:
bitacora['protocol'].value_counts()

HTTP/1.1    241991
HTTP/1.0     19840
Name: protocol, dtype: int64

In [54]:
bitacora['status'].value_counts()

200    226382
304     25269
404      8630
301       870
206       304
403       194
302       109
502        44
408        37
400        26
401         5
500         3
Name: status, dtype: int64

In [55]:
bitacora['referer'].value_counts().head(25)

http://www.akumenius.com/                                                                                                                                      41394
http://www.akumenius.com/hotel-list                                                                                                                            22039
http://test.akumenius.com/newdesign/                                                                                                                           11839
http://www.akumenius.com/hoteles-baratos/hoteles-todo-incluido.html                                                                                             5398
http://www.akumenius.com/chollos                                                                                                                                4390
http://www.akumenius.com/escapadas                                                                                                                              3708
http://www

In [56]:
bitacora['user_agent'].value_counts().head(25)

Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)                                                                      50914
Mozilla/5.0 (X11; U; Linux i686; ca; rv:1.9.2.17) Gecko/20110428 Fedora/3.6.17-1.fc13 Firefox/3.6.17                                          13920
Apache (internal dummy connection)                                                                                                            13892
Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/33.0.1750.117 Safari/537.36                                  9388
Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/33.0.1750.117 Safari/537.36                                         7761
Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko                                                                           7081
Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/33.0.1750.117 Safari/537.36          

9. Per a la columna size que conté dades numèriques presento un petit resum estadístic:

In [57]:
bitacora['size'].max()

45564687.0

In [58]:
bitacora['size'].mean()

11827.712901638895

In [59]:
bins = [0, 10000, 20000, 30000, 50000000]
x=pd.cut(bitacora['size'],bins)
pd.value_counts(x)

(0, 10000]           166342
(10000, 20000]        32053
(30000, 50000000]     12539
(20000, 30000]         8604
Name: size, dtype: int64

***Exercici 3***: Geolocalitza les IP's.

Primer faig una recerca per a explorar quines alternatives hi ha per geolocalitzar adreces IP amb python. Trio una alternativa que es el paquet *ip2geotools* (https://pypi.org/project/ip2geotools/)

L'ús bàsic es generar un objecte amb DbIpCity.get(*'adreça_ip'*, api_key='free')

Segueixo les instruccions del petit tutorial per a familiaritzar-me amb el seu funcionament:

In [60]:
from ip2geotools.databases.noncommercial import DbIpCity
response = DbIpCity.get('147.229.2.90', api_key='free')

In [61]:
response.city

'Brno střed'

In [62]:
response.region

'South Moravian'

In [63]:
response.country

'CZ'

In [64]:
response.latitude

49.1819648

In [65]:
response.longitude

16.610504

Genero una nova dataframe amb les IP's que encapçalen el numero de visites, a la qual hi afegiré les dades de geolocalització. Una altra alternativa seria ficar aquestes dades per a cada registre de la dataframe 'bitacora', pero no ho faig perque té un cost molt elevat de temps el fet de cridar 261873 vegades una petició de ip2geotools.

In [66]:
ip_geolocation = bitacora["ip_client"].value_counts().rename_axis('ip').reset_index(name="visits")

ip_geolocation=ip_geolocation[ip_geolocation["visits"]>500]
ip_geolocation


,ip,visits
0,66.249.76.216,46382
1,80.28.221.123,14725
2,127.0.0.1,13892
3,217.125.71.222,5201
4,66.249.75.148,3558
5,162.243.192.191,2927
6,62.117.197.230,2567
7,89.128.176.162,1093
8,198.143.133.154,1045
9,176.31.255.177,1044


A continuació, utilitzo la llibreria ip2geotools per a geolocalitzar aquestes IP's. Assignaré tota aquesta informació com a columnes addicionals al dataframe 'ip_geolocation'. Per si un cas hi ha resultats nuls al obtenir les propietats d'aquesta llibreria creo unes funcions que retornin un Nan en aquest cas. Aquestes funcions son les que aplico amb el mètode .apply()

Després presento una visualització de les dades. L'adreça 127.0.0.1 és la que em dona un None perque per defecte és el local host.

In [67]:

def ip_ciutat(ip):
    try:
        return DbIpCity.get(ip,api_key='free').city
    except:
        return np.nan
def ip_regio(ip):
    try:
        return DbIpCity.get(ip,api_key='free').region
    except:
        return np.nan
def ip_pais(ip):
    try:
        return DbIpCity.get(ip,api_key='free').country
    except:
        return np.nan
def ip_latitut(ip):
    try:
        return DbIpCity.get(ip,api_key='free').latitude
    except:
        return np.nan
def ip_longitut(ip):
    try:
        return DbIpCity.get(ip,api_key='free').longitude
    except:
        return np.nan

ip_geolocation["city"]=ip_geolocation.loc[:,'ip'].apply(ip_ciutat)
ip_geolocation["region"]=ip_geolocation.loc[:,'ip'].apply(ip_regio)
ip_geolocation["country"]=ip_geolocation.loc[:,'ip'].apply(ip_pais)       
ip_geolocation["latitude"]=ip_geolocation.loc[:,'ip'].apply(ip_latitut)
ip_geolocation["longitude"]=ip_geolocation.loc[:,'ip'].apply(ip_longitut)


In [34]:
ip_geolocation

,ip,visits,city,region,country,latitude,longitude
0,66.249.76.216,46382,Mountain View,California,US,37.389389,-122.083210
1,80.28.221.123,14725,Madrid,Madrid,ES,40.416705,-3.703582
2,127.0.0.1,13892,None,None,ZZ,36.733438,-119.833235
3,217.125.71.222,5201,Madrid,Madrid,ES,40.416705,-3.703582
4,66.249.75.148,3558,Mountain View,California,US,37.389389,-122.083210
5,162.243.192.191,2927,New York,New York,US,40.712728,-74.006015
6,62.117.197.230,2567,Madrid,Madrid,ES,40.416705,-3.703582
7,89.128.176.162,1093,Pozuelo de Alarcón,Madrid,ES,40.434653,-3.814834
8,198.143.133.154,1045,Chicago (Loop),Illinois,US,42.021578,-88.183000
9,176.31.255.177,1044,Roubaix,Hauts-de-France,FR,50.691589,3.174173
